In [12]:
from datasets import load_dataset
from qdrant_client.http import models as rest

In [9]:
from qdrant_client import QdrantClient
from dotenv import load_dotenv
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os

In [2]:
dataset = load_dataset("pubmed_qa", "pqa_labeled")

In [4]:
import cohere

COHERE_API_KEY = "KqApPE9kQ9TSKjc8OkD0X9W2YEJlCkQBmg1Xb1tv"
cohere_client = cohere.Client(COHERE_API_KEY)

In [5]:
# Generating the embeddings with Cohere client library
embeddings = cohere_client.embed(
    texts=["A test sentence"],
    model="large",
)
vector_size = len(embeddings.embeddings[0])
print(vector_size) # output: 4096

unknown field: parameter compress is not a valid field


4096


In [7]:
len(embeddings.embeddings[0])

4096

In [8]:
answer_response = cohere_client.embed(
    texts=dataset["train"]["long_answer"],
    model="large",
)
vectors = [
    # Conversion to float is required for Qdrant
    list(map(float, vector)) 
    for vector in answer_response.embeddings
]
ids = [entry["pubid"] for entry in dataset["train"]]

unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field


unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field


In [10]:
load_dotenv()

qdrant_url = os.getenv('QDRANT_URL')

client = QdrantClient(url=qdrant_url)


In [11]:
client.create_collection(
    collection_name="pubmed_qa_cohere_4096",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

True

In [13]:
client.upsert(
    collection_name="pubmed_qa_cohere_4096", 
    points=rest.Batch(
        ids=ids[0:100],
        vectors=vectors[0:100],
        payloads=list(dataset["train"])[:100],
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
dataset["train"]["question"]

['Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'Landolt C and snellen e acuity: differences in strabismus amblyopia?',
 'Syncope during bathing in infants, a pediatric form of water-induced urticaria?',
 'Are the long-term results of the transanal pull-through equal to those of the transabdominal pull-through?',
 'Can tailored interventions increase mammography use among HMO women?',
 'Double balloon enteroscopy: is it efficacious and safe in a community setting?',
 '30-Day and 1-year mortality in emergency general surgery laparotomies: an area of concern and need for improvement?',
 'Is adjustment for reporting heterogeneity necessary in sleep disorders?',
 'Do mutations causing low HDL-C promote increased carotid intima-media thickness?',
 "A short stay or 23-hour ward in a general and academic children's hospital: are they effective?",
 "Did Chile's traffic law reform push police enforcement?",
 'Therapeutic anticoagulation in the tra

In [16]:
question_response = cohere_client.embed(
    texts=dataset["train"]["question"],
    model="large",
)

unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field
unknown field: parameter compress is not a valid field


In [18]:
import tqdm

# Finding the position at which Qdrant provided the expected answer for each question. 
# That allows to calculate accuracy@k for different values of k.
k_max = 10
answer_positions = []
for embedding, pubid in tqdm.tqdm(zip(question_response.embeddings, ids)):
    response = client.search(
        collection_name="pubmed_qa_cohere_4096",
        query_vector=embedding,
        limit=k_max,
    )

    answer_ids = [record.id for record in response]
    if pubid in answer_ids:
        answer_positions.append(answer_ids.index(pubid))
    else:
        answer_positions.append(-1)

1000it [00:03, 268.06it/s]


In [19]:
# Prepared answer positions are being used to calculate different values of accuracy@k
for k in range(1, k_max + 1):
    correct_answers = len(
        list(
            filter(lambda x: 0 <= x < k, answer_positions)
        )
    )
    print(f"accuracy@{k} =", correct_answers / len(dataset["train"]))

accuracy@1 = 0.094
accuracy@2 = 0.095
accuracy@3 = 0.095
accuracy@4 = 0.097
accuracy@5 = 0.098
accuracy@6 = 0.098
accuracy@7 = 0.098
accuracy@8 = 0.099
accuracy@9 = 0.099
accuracy@10 = 0.099
